In [1]:
import numpy as np
import pandas as pd
import os, shutil, json
import matplotlib.pyplot as plt
import keras
from PIL import Image
from pathlib import Path
from statistics import mean 
from keras import backend as K
from keras.models import Sequential, load_model
from keras.optimizers import RMSprop, Adam, SGD
from keras.preprocessing import image
from keras.utils.np_utils import to_categorical
from keras.losses import categorical_crossentropy, binary_crossentropy, mean_squared_error, mean_squared_logarithmic_error
from keras.preprocessing.image import img_to_array, ImageDataGenerator
from keras import layers, models, regularizers, backend, utils
from keras.layers import Activation, BatchNormalization, Conv2D, Conv2DTranspose, Dense, Dropout, Flatten, initializers, InputSpec, Input, LeakyReLU, MaxPooling2D, Reshape

abs_path = r'%s' % os.getcwd().replace('\\','/')

Using TensorFlow backend.
C:\Users\Pichau\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Pichau\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Pichau\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Pichau\Anaconda3\lib\site-packages\tensorflow\python\framework\dtype

# Modelo Genérico

In [2]:
class rede:
    def __init__(self, batch_size = 20, images_size = 64):
        self.generator_input = 100
        self.latent_dim = 32
        self.height = images_size
        self.width = images_size
        self.channels = 3
        self.iterations = 20000
        self.batch_size = batch_size
        self.NPZ_file_path = os.path.join(abs_path, "2_imagens_Mckinsey666_"+str(self.height)+"_"+str(self.height)+".npz").replace('\\','/')
        self.create_numpy_file()
        self.my_ReduceLROnPlateau_losses = []
        self.my_ReduceLROnPlateau_sentinela = 0
        self.generator = []
        self.discriminator = []
        self.gan = []
        
        images_path = os.path.join(abs_path,'2_Generated_Images_' + str(self.height) + '_' + str(self.height) + '_Presentation').replace('\\','/')
        if not os.path.exists(images_path):
            os.mkdir(images_path)
        self.save_dir = images_path
        
    def create_numpy_file(self):
        imagens_x = []
        if not os.path.exists(self.NPZ_file_path):
            Imagens_DataFrame = pd.read_csv("./1_imagens_Mckinsey666.csv")
            for index, row in Imagens_DataFrame.iterrows():
                    image_path = os.path.join(abs_path, '1_Mckinsey666', row['path']).replace('\\','/')
                    image = Image.open(image_path)
                    newsize = (self.height, self.width)
                    image = image.resize(newsize, Image.LANCZOS) 
                    image_numpy = np.asarray(image)
                    imagens_x.append(image_numpy)
                    #imagens_x.append(np.rot90(image_numpy))
                    #imagens_x.append(np.rot90(image_numpy, 3))

            imagens_x = np.asarray(imagens_x, dtype=np.float32)
            np.savez(self.NPZ_file_path, x=imagens_x)
    
    def get_images_as_nparray(self):
        with np.load(self.NPZ_file_path) as data:
            imagens_x = data['x']
        #x_train = imagens_x.reshape((imagens_x.shape[0],) + (self.height, self.width, self.channels)).astype('float32') / 255.
        x_train = (imagens_x.reshape((imagens_x.shape[0],) + (self.height, self.width, self.channels)).astype('float32') - 127.5 ) / 127.5
        #x_train = (imagens_x.reshape((imagens_x.shape[0],) + (self.height, self.width, self.channels)).astype('float32') / 127.5 ) - 0.5
        return x_train
        
    def create_generator(self):
        generator_input = keras.Input(shape=(self.generator_input,))
        
        x = Dense(128 * int(self.height/2) * int(self.width/2))(generator_input)
        x = LeakyReLU()(x)
        x = Reshape((int(self.height/2), int(self.width/2), 128))(x)

        x = Conv2D(256, 5, padding='same')(x)
        x = LeakyReLU()(x)

        x = Conv2DTranspose(256, 4, strides=2, padding='same')(x)
        x = LeakyReLU()(x)
        x = Conv2D(256, 5, padding='same')(x)
        x = LeakyReLU()(x)
        x = Conv2D(256, 5, padding='same')(x)
        x = LeakyReLU()(x)
        #Produces a 32 × 32 1-channel feature map (shape of a CIFAR10 image)
        x = Conv2D(self.channels, 7, activation='tanh', padding='same')(x)
        #Instantiates the generator model, which maps the input of shape (latent_dim,) into an image of shape (32, 32, 3)
        model = keras.models.Model(generator_input, x)
        
        return model
        
    
    def create_discriminator(self):
        discriminator_input = Input(shape=(self.height, self.width, self.channels))
        x = Conv2D(128, 3)(discriminator_input)
        x = LeakyReLU()(x)

        x = Conv2D(128, 4, strides=2)(x)
        x = LeakyReLU()(x)

        x = Conv2D(128, 4, strides=2)(x)
        x = LeakyReLU()(x)

        x = Conv2D(128, 4, strides=2)(x)
        x = LeakyReLU()(x)

        x = Flatten()(x)
        x = Dropout(0.4)(x)
        x = Dense(1, activation='sigmoid')(x)
        model = keras.models.Model(discriminator_input, x)

        discriminator_optimizer = keras.optimizers.RMSprop(
                lr=0.0008,
                clipvalue=1.0, #Uses gradient clipping (by value) in the optimizer
                decay=1e-8) #To stabilize training, uses learning-rate decay
        model.compile(optimizer=Adam(0.0001, beta_1=0.5, beta_2=0.9), loss='binary_crossentropy')
        #model.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')
        return model
    
    def create_adversarial_net(self):
        #Sets discriminator weights to non-trainable (this will only apply to the gan model) 
        self.discriminator = self.create_discriminator()
        self.generator = self.create_generator()
        
        self.discriminator.trainable = False
        gan_input = keras.Input(shape=(self.generator_input,))
        gan_output = self.discriminator(self.generator(gan_input))
        self.gan = keras.models.Model(gan_input, gan_output)
        gan_optimizer = RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
        #self.gan.compile(optimizer=gan_optimizer, loss="binary_crossentropy")
        self.gan.compile(optimizer=Adam(0.0001, beta_1=0.5, beta_2=0.9), loss='binary_crossentropy')

    def my_ReduceLROnPlateau_loss(self, new_loss, step_of_iterations, factor = 0.5, patience = 10, epsilon = 1e-04, cooldown = 0, min_lr=0):
        #Se a lista estiver cheia, remove o primeiro elemento
        if len(self.my_ReduceLROnPlateau_losses) == patience:
            self.my_ReduceLROnPlateau_losses.pop(0)
            
        #Se for lista, existirá dois valores dentro, então  obtém só o primeiro
        if type(new_loss) is list:
            new_loss = new_loss[0]
        self.my_ReduceLROnPlateau_losses.append(new_loss)
        
        valor_antigo = float(K.get_value(self.discriminator.optimizer.lr))
        #Checa se a lista de pesos está cheia, a quantas iterações foi a última alteração
        if len(self.my_ReduceLROnPlateau_losses) == 10 and (step_of_iterations - self.my_ReduceLROnPlateau_sentinela) > cooldown and valor_antigo > min_lr:
            valor_min = mean(self.my_ReduceLROnPlateau_losses) - epsilon
            valor_max = mean(self.my_ReduceLROnPlateau_losses) + epsilon
            #verifica se houve alterações nos "Losses" nas últimas "patience" épocas
            validacao = [ (loss < valor_min or loss > valor_max) for loss in self.my_ReduceLROnPlateau_losses]
            if False in validacao: 
                valor_novo = valor_antigo * factor
                K.set_value(self.discriminator.optimizer.lr, valor_novo)
                print("[CALLBACK] Learning Rate atualizado na época {} de {} para {}".format(step_of_iterations, valor_antigo, valor_novo))
                #Se houver uma alteração, esprar "cooldown" rodadas para fazer uma nova alteração
                self.my_ReduceLROnPlateau_sentinela = step_of_iterations
                
    def wasserstein_loss(self, y_true, y_pred):
        return K.mean(y_true * y_pred)
    
    def get_randomly_batch_images(self, x_train):
        pacotes = x_train.shape[0] // self.batch_size
        inicio_batch = int(pacotes * np.random.randint(self.batch_size + 1, size=1))
        final_batch = int(inicio_batch + self.batch_size)
        if final_batch > x_train.shape[0]:
            diferenca = final_batch - x_train.shape[0]
            real_images = np.concatenate((x_train[inicio_batch:], x_train[:diferenca]), axis=0)
            #print("{} {} {} {} {} {} {}".format(inicio_batch, final_batch, x_train.shape[0], x_train[inicio_batch:].shape, x_train[:diferenca].shape, diferenca, real_images.shape))
        else:
            real_images = x_train[inicio_batch:final_batch]
        assert real_images.shape == (self.batch_size, self.height, self.height, self.channels)
        return real_images
    
    def build_network(self):
        start = 0
        self.create_adversarial_net()
        x_train = self.get_images_as_nparray()
        
        for step in range(self.iterations):            
            #TREINAR O DISCRIMINATOR
            random_latent_vectors = np.random.normal(size=(self.batch_size, self.generator_input))
            generated_images = self.generator.predict(random_latent_vectors)

            stop = start + self.batch_size
            real_images = x_train[start: stop]

            #real_images = self.get_randomly_batch_images(x_train)
            combined_images = np.concatenate([generated_images, real_images])
            labels = np.concatenate([np.zeros((self.batch_size, 1), dtype=np.float32), np.ones((self.batch_size, 1), dtype=np.float32)])
            #labels = np.concatenate([-np.ones((self.batch_size, 1)), np.ones((self.batch_size, 1))])
            #Adds random noise to the labels—an important trick!
            labels += 0.05 * np.random.random(labels.shape)
            d_loss = self.discriminator.train_on_batch(combined_images, labels)
            
            #TREINAR O GENERATOR
            random_latent_vectors = np.random.normal(size=(self.batch_size, self.generator_input))
            misleading_targets = np.ones((self.batch_size, 1), dtype=np.float32)
            a_loss = self.gan.train_on_batch(random_latent_vectors, misleading_targets)
            
            #CALLBACKS
            self.my_ReduceLROnPlateau_loss(d_loss, step, patience=15, cooldown = 15)

            start += self.batch_size
            if start > len(x_train) - self.batch_size:
                start = 0

            if step % 25== 0 or step == self.iterations-1:
                self.generator.save("2_Generator_model_gan_"+str(self.height)+"_"+str(self.height)+"_Presentation.h5")
                print('{} - discriminator loss: {} - adversarial loss: {}'.format(step, d_loss, a_loss))
                step_string = "{:06d}".format(step)
                new_dir = os.path.join(self.save_dir,'Epoca_'+step_string).replace('\\','/')
                if not os.path.exists(new_dir):
                    os.mkdir(new_dir)
                for image_number in range(self.batch_size):
                    img = image.array_to_img(np.squeeze(np.round((generated_images[image_number] * 127.5) + 127.5).astype(np.uint8)), scale=False)
                    img.save(os.path.join(new_dir,'2_GAN_' + str(image_number) + '_generated_anime.png'))

In [3]:
rede = rede(64, 64)
rede.build_network()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


C:\Users\Pichau\Anaconda3\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 - discriminator loss: 0.7068132758140564 - adversarial loss: 0.6931306719779968
25 - discriminator loss: 0.2642636299133301 - adversarial loss: 0.7125013470649719
50 - discriminator loss: 1.1692270040512085 - adversarial loss: 0.29036465287208557
75 - discriminator loss: 0.6987295150756836 - adversarial loss: 0.7806828022003174
100 - discriminator loss: 0.718989372253418 - adversarial loss: 0.8321815729141235
125 - discriminator loss: 0.6619747877120972 - adversarial loss: 0.6345105171203613
150 - discriminator loss: 0.5410618782043457 - adversarial loss: 1.4828530550003052
175 - discriminator loss: 0.7011469006538391 - adversarial loss: 0.9433771371841431
200 - discriminator loss: 0.550859808921814 - adversarial loss: 0.9601483345031738
225 - discriminator loss: 0.5943605899810791 - adversarial loss: 1.1185613870620728
250 - discriminator loss: 0.5534605383872986 - adversarial loss: 1.2851629257202148
275 - discriminator loss: 0.6759033799171448 - adversarial loss: 0.810902237892150

2450 - discriminator loss: 0.6869555115699768 - adversarial loss: 0.6782206296920776
2475 - discriminator loss: 0.6953171491622925 - adversarial loss: 0.6966924667358398
2500 - discriminator loss: 0.6823137998580933 - adversarial loss: 0.6192054748535156
2525 - discriminator loss: 0.6946934461593628 - adversarial loss: 0.6739760637283325
2550 - discriminator loss: 0.6915527582168579 - adversarial loss: 0.6329765319824219
2575 - discriminator loss: 0.6910058259963989 - adversarial loss: 0.6638435125350952
2600 - discriminator loss: 0.7095014452934265 - adversarial loss: 0.6072115898132324
2625 - discriminator loss: 0.7023853063583374 - adversarial loss: 0.5993171334266663
2650 - discriminator loss: 0.6936624050140381 - adversarial loss: 0.634710431098938
2675 - discriminator loss: 0.6888272762298584 - adversarial loss: 0.6888778805732727
2700 - discriminator loss: 0.6918286085128784 - adversarial loss: 0.6521841287612915
2725 - discriminator loss: 0.683228075504303 - adversarial loss: 0

4875 - discriminator loss: 0.6885538101196289 - adversarial loss: 0.6777774095535278
4900 - discriminator loss: 0.6903725862503052 - adversarial loss: 0.6913822889328003
4925 - discriminator loss: 0.6983163356781006 - adversarial loss: 0.6677148938179016
4950 - discriminator loss: 0.6873525381088257 - adversarial loss: 0.6469400525093079
4975 - discriminator loss: 0.6966453194618225 - adversarial loss: 0.6563756465911865
5000 - discriminator loss: 0.6980607509613037 - adversarial loss: 0.6858335733413696
5025 - discriminator loss: 0.7046037912368774 - adversarial loss: 0.6310698986053467
5050 - discriminator loss: 0.7108476161956787 - adversarial loss: 0.6882312893867493
5075 - discriminator loss: 0.689849317073822 - adversarial loss: 0.6548662185668945
5100 - discriminator loss: 0.692882776260376 - adversarial loss: 0.6935901641845703
5125 - discriminator loss: 0.7019450664520264 - adversarial loss: 0.660699188709259
5150 - discriminator loss: 0.6943239569664001 - adversarial loss: 0.

...

9725 - discriminator loss: 0.6945909261703491 - adversarial loss: 0.6811840534210205
9750 - discriminator loss: 0.697429895401001 - adversarial loss: 0.6478967666625977
9775 - discriminator loss: 0.6854397058486938 - adversarial loss: 0.6461321115493774
9800 - discriminator loss: 0.7019248008728027 - adversarial loss: 0.6677196025848389
9825 - discriminator loss: 0.6893069744110107 - adversarial loss: 0.6555277109146118
9850 - discriminator loss: 0.6956192851066589 - adversarial loss: 0.6516340970993042
9875 - discriminator loss: 0.6987451314926147 - adversarial loss: 0.6379444599151611
9900 - discriminator loss: 0.7173001170158386 - adversarial loss: 0.6345551013946533
9925 - discriminator loss: 0.6953607797622681 - adversarial loss: 0.6471492052078247
9950 - discriminator loss: 0.6983697414398193 - adversarial loss: 0.6678364276885986
9975 - discriminator loss: 0.6863130331039429 - adversarial loss: 0.6643017530441284
10000 - discriminator loss: 0.6993637681007385 - adversarial loss:

12125 - discriminator loss: 0.6894451379776001 - adversarial loss: 0.6713483333587646
12150 - discriminator loss: 0.6956729888916016 - adversarial loss: 0.6582602262496948
12175 - discriminator loss: 0.6920897960662842 - adversarial loss: 0.6640939712524414
12200 - discriminator loss: 0.6979538202285767 - adversarial loss: 0.6765854358673096
12225 - discriminator loss: 0.6957396268844604 - adversarial loss: 0.6458852291107178
12250 - discriminator loss: 0.6905272006988525 - adversarial loss: 0.654316782951355
12275 - discriminator loss: 0.7014744281768799 - adversarial loss: 0.6552292108535767
12300 - discriminator loss: 0.6952146291732788 - adversarial loss: 0.6656909584999084
12325 - discriminator loss: 0.6865420341491699 - adversarial loss: 0.6479511260986328
12350 - discriminator loss: 0.6889199018478394 - adversarial loss: 0.6635236740112305
12375 - discriminator loss: 0.6890971660614014 - adversarial loss: 0.6767945289611816
12400 - discriminator loss: 0.6885914206504822 - advers

14525 - discriminator loss: 0.696887195110321 - adversarial loss: 0.6535478830337524
14550 - discriminator loss: 0.6925787925720215 - adversarial loss: 0.6630197167396545
14575 - discriminator loss: 0.6933413743972778 - adversarial loss: 0.6633814573287964
14600 - discriminator loss: 0.6911982297897339 - adversarial loss: 0.6413143873214722
14625 - discriminator loss: 0.6913349628448486 - adversarial loss: 0.638353705406189
14650 - discriminator loss: 0.6938955783843994 - adversarial loss: 0.6618245840072632
14675 - discriminator loss: 0.6878018379211426 - adversarial loss: 0.6896710991859436
14700 - discriminator loss: 0.6956202387809753 - adversarial loss: 0.6570423245429993
14725 - discriminator loss: 0.6970152258872986 - adversarial loss: 0.6652165055274963
14750 - discriminator loss: 0.6896001100540161 - adversarial loss: 0.6444785594940186
14775 - discriminator loss: 0.6949611902236938 - adversarial loss: 0.6474847197532654
14800 - discriminator loss: 0.6922687888145447 - adversa

16925 - discriminator loss: 0.693689227104187 - adversarial loss: 0.6145866513252258
16950 - discriminator loss: 0.6911500692367554 - adversarial loss: 0.6317548751831055
16975 - discriminator loss: 0.6945136189460754 - adversarial loss: 0.6532210111618042
17000 - discriminator loss: 0.6870113015174866 - adversarial loss: 0.6463987827301025
17025 - discriminator loss: 0.6896746754646301 - adversarial loss: 0.6213029623031616
17050 - discriminator loss: 0.6902108192443848 - adversarial loss: 0.6757251620292664
17075 - discriminator loss: 0.6942070722579956 - adversarial loss: 0.6314942240715027
17100 - discriminator loss: 0.695408821105957 - adversarial loss: 0.6628881692886353
17125 - discriminator loss: 0.6945217847824097 - adversarial loss: 0.6476518511772156
17150 - discriminator loss: 0.6943165063858032 - adversarial loss: 0.6503602862358093
17175 - discriminator loss: 0.6918833255767822 - adversarial loss: 0.6334880590438843
17200 - discriminator loss: 0.6937310099601746 - adversa

19325 - discriminator loss: 0.6957308053970337 - adversarial loss: 0.6460227966308594
19350 - discriminator loss: 0.6917450428009033 - adversarial loss: 0.664583683013916
19375 - discriminator loss: 0.6908226609230042 - adversarial loss: 0.6389786601066589
19400 - discriminator loss: 0.6945744752883911 - adversarial loss: 0.6255890130996704
19425 - discriminator loss: 0.698574960231781 - adversarial loss: 0.6681845188140869
19450 - discriminator loss: 0.6918778419494629 - adversarial loss: 0.6224232316017151
19475 - discriminator loss: 0.6906567811965942 - adversarial loss: 0.6602786779403687
19500 - discriminator loss: 0.696316123008728 - adversarial loss: 0.6529629230499268
19525 - discriminator loss: 0.6922343969345093 - adversarial loss: 0.649630069732666
19550 - discriminator loss: 0.6938008069992065 - adversarial loss: 0.6408176422119141
19575 - discriminator loss: 0.6922183036804199 - adversarial loss: 0.6410043239593506
19600 - discriminator loss: 0.6915130615234375 - adversari

In [4]:
def test():
    model = load_model('2_Generator_model_gan_64_64.h5')
    batch_size = 128
    random_latent_vectors = np.random.normal(size=(batch_size, 100))
    generated_images = model.predict(random_latent_vectors)
    images_path = os.path.join(abs_path,'2_Generated_Images_Test_64_64').replace('\\','/')
    if not os.path.exists(images_path):
        os.mkdir(images_path)
    for index in range(batch_size):
        img = image.array_to_img(np.squeeze(np.round((generated_images[index] * 127.5) + 127.5).astype(np.uint8)), scale=False)
        img.save(os.path.join(images_path,'2_GAN_' + str(index+1) + '_generated_anime.png'))
    print("OI")
#test()

In [5]:
print("Oi")

Oi
